In [16]:
from datasets import load_dataset
from fast_langdetect import detect
import random

NUM_PROC=20

In [ ]:
wikisource_preferences_ru = load_dataset("kristaller486/wikisource_preferences_ru", split='train')
def f(e):
    return int(e['gpt-4.1-mini-orig-segment-score']) >= 4

def to_json_ready(ds):
    json_ready = []
    for conv in ds['messages']:
        conv_to_json = []
        fixed_conv = conv
        for turn in fixed_conv:
            fixed_content = turn['content']
            conv_to_json.append(
                {
                    "role": turn['role'],
                    "content": fixed_content
                }
            )
        json_ready.append(conv_to_json)
    return json_ready
    
wikisource_preferences_ru = wikisource_preferences_ru.filter(f, batched=False, num_proc=NUM_PROC)
wikisource_preferences_ru = to_json_ready(wikisource_preferences_ru)
wikisource_preferences_ru[:2]

In [ ]:
kalo_misc_part2_no_system_ru = load_dataset('Arketov/kalo_misc_part2_no_system_ru', split='train')
def f(e):
    from fast_langdetect import detect
    for turn in e["conversation"]:
        if turn['role'] != 'user':
            continue
        if detect(turn['content'], model="full")[0]['lang'] != 'ru':
            return False
    return True

def to_json_ready(ds):
    json_ready = []
    for conv in ds['conversation']:
        conv_to_json = []
        fixed_conv = conv[:-1]
        for turn in fixed_conv:
            fixed_content = turn['content']
            if fixed_content.startswith('"'):
                fixed_content = fixed_content[1:]
            if fixed_content.endswith('"'):
                fixed_content = fixed_content[:-1]
            conv_to_json.append(
                {
                    "role": turn['role'],
                    "content": fixed_content
                }
            )
        json_ready.append(conv_to_json)
    return json_ready
    

kalo_misc_part2_no_system_ru = kalo_misc_part2_no_system_ru.filter(f, batched=False, num_proc=NUM_PROC)
kalo_misc_part2_no_system_ru = to_json_ready(kalo_misc_part2_no_system_ru)
kalo_misc_part2_no_system_ru[:2]

In [ ]:
writingprompts_ru = load_dataset("kristaller486/writingprompts-ru", split='train')
def f(e):
    return True

def to_json_ready(ds):
    json_ready = []
    for prompt in ds['prompt']:
        fixed_content = 'Напиши текст по этой подсказке: ' + prompt
        conv_to_json = []
        conv_to_json.append(
            {
                "role": 'user',
                "content": fixed_content
            }
        )
        json_ready.append(conv_to_json)
    return json_ready
    
writingprompts_ru = writingprompts_ru.filter(f, batched=False, num_proc=NUM_PROC)
writingprompts_ru = to_json_ready(writingprompts_ru)
writingprompts_ru[:2]

In [ ]:
twix = load_dataset("t-tech/T-Wix", split='train')
def f(e):
    return e['subset'] == 'general'

def to_json_ready(ds):
    json_ready = []
    for conv in ds['messages']:
        conv_to_json = []
        fixed_conv = conv[:-1]
        for turn in fixed_conv:
            fixed_content = turn['content']
            conv_to_json.append(
                {
                    "role": turn['role'],
                    "content": fixed_content
                }
            )
        json_ready.append(conv_to_json)
    return json_ready
    
twix = twix.filter(f, batched=False, num_proc=NUM_PROC)
twix = to_json_ready(twix)
twix[:2]

In [18]:
random.seed(42)

lite_bench_100 = []
base_bench_250 = []
large_bench_500 = []

lite_bench_100.extend(random.sample(wikisource_preferences_ru, 16))
lite_bench_100.extend(random.sample(kalo_misc_part2_no_system_ru, 17))
lite_bench_100.extend(random.sample(writingprompts_ru, 17))
lite_bench_100.extend(random.sample(twix, 50))

base_bench_250.extend(random.sample(wikisource_preferences_ru, 41))
base_bench_250.extend(random.sample(kalo_misc_part2_no_system_ru, 42))
base_bench_250.extend(random.sample(writingprompts_ru, 42))
base_bench_250.extend(random.sample(twix, 125))

large_bench_500.extend(random.sample(wikisource_preferences_ru, 83))
large_bench_500.extend(random.sample(kalo_misc_part2_no_system_ru, 84))
large_bench_500.extend(random.sample(writingprompts_ru, 84))
large_bench_500.extend(random.sample(twix, 250))

In [20]:
import json

with open('lite_bench_100.json', 'w', encoding='utf-8') as f:
    json.dump(lite_bench_100, f, ensure_ascii=False, indent=4)

with open('base_bench_250.json', 'w', encoding='utf-8') as f:
    json.dump(base_bench_250, f, ensure_ascii=False, indent=4)

with open('large_bench_500.json', 'w', encoding='utf-8') as f:
    json.dump(large_bench_500, f, ensure_ascii=False, indent=4)